In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import random
import statistics
from scipy.stats import cauchy

def de(fobj, bounds, mut=0.9, cr=0.1, popsize=20, its=1000, goal = 0):
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    population = min_b + pop * diff
    fitness = np.asarray([fobj(ind) for ind in population])
    best_idx = np.argmin(fitness)
    best = population[best_idx]
    for i in range(its):
        for j in range(popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c = population[np.random.choice(idxs, 3, replace = False)]
            mutant = np.clip(a + mut * (b - c), min_b, max_b)
            cross_points = np.random.rand(dimensions) < cr
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, population[j])
            f = fobj(trial)
            if f < fitness[j]:
                fitness[j] = f
                population[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial  
        if(np.fabs(min(fitness) - goal) < 1e-6):
            print(i)
            break
        yield best, fitness[best_idx]
        pass
    pass


def de_randtobest_1(fobj, bounds, mut=0.5, cr=0.3, popsize=20, its=1000, goal = 0):
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    population = min_b + pop * diff
    fitness = np.asarray([fobj(ind) for ind in population])
    best_idx = np.argmin(fitness)
    best = population[best_idx]
    for i in range(its):
        for j in range(popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b= population[np.random.choice(idxs, 2, replace = False)]
            mutant = np.clip(population[j] + mut * (best - population[j]) + mut * (a - b), min_b, max_b)
            cross_points = np.random.rand(dimensions) < cr
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, population[j])
            f = fobj(trial)
            if f < fitness[j]:
                fitness[j] = f
                population[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial  
        if(np.fabs(min(fitness) - goal) < 1e-6):
            print(i)
            break
        yield best, fitness[best_idx]
        pass
    pass


def de_randtobest_2(fobj, bounds, mut=0.5, cr=0.3, popsize=20, its=1000, goal = 0):
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    population = min_b + pop * diff
    fitness = np.asarray([fobj(ind) for ind in population])
    best_idx = np.argmin(fitness)
    best = population[best_idx]
    for i in range(its):
        for j in range(popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c, d = population[np.random.choice(idxs, 4, replace = False)]
            mutant = np.clip(population[j] + mut * (best - population[j]) + mut * (a - b) + mut * (c - d), min_b, max_b)
            cross_points = np.random.rand(dimensions) < cr
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, population[j])
            f = fobj(trial)
            if f < fitness[j]:
                fitness[j] = f
                population[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial  
        if(np.fabs(min(fitness) - goal) < 1e-6):
            print(i)
            break
        yield best, fitness[best_idx]
        pass
    pass


def jde(fobj, bounds, mut=0.5, cr=0.9, popsize=20, its=1000, goal = 0):
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    population = min_b + pop * diff
    fitness = np.asarray([fobj(ind) for ind in population])
    best_idx = np.argmin(fitness)
    best = population[best_idx]
    for i in range(its):
        for j in range(popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c = population[np.random.choice(idxs, 3, replace = False)]
            mutant = np.clip(a + mut * (b - c), min_b, max_b)
            cross_points = np.random.rand(dimensions) < cr
            randj = np.random.rand(4)
            if randj[0]<0.1:
                mut = 0.1+randj[1]*0.9
            if randj[2]<0.1:
                crossp = randj[3]
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, population[j])
            f = fobj(trial)
            if f < fitness[j]:
                fitness[j] = f
                population[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial  
        if(np.fabs(min(fitness) - goal) < 1e-6):
            print(i)
            break
        yield best, fitness[best_idx]
        pass
    pass

def sade(fobj, bounds, popsize=20, its=1000, goal = 0):
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    population = min_b + pop * diff
    fitness = np.asarray([fobj(ind) for ind in population])
    best_idx = np.argmin(fitness)
    best = population[best_idx]
    sp = strategy_probability = [0.25] * 4
    lp = learning_period = 5
    success_memory = np.zeros([lp, 4])
    failure_memory = np.zeros([lp, 4])
    cr_memory = [[],[],[],[]]
    for i in range (lp):
        for j in range (popsize):
            popj = population[j]
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c, d, e = population[np.random.choice(idxs, 5, replace = False)]
            strategy_num = -1
            mut = random.gauss(0.5, 0.3)
            cr = random.gauss(0.5, 0.1)
            if((cr < 0) or (cr > 1)):
                cr = random.gauss(0.5, 0.1)
            rand_sp = np.random.rand()
            if(rand_sp < sp[0]):
                strategy_num = 0
                trial = rand_1_bin(a, b, c, mut, min_b, max_b, popj, dimensions, cr)
            elif(rand_sp < sum(sp[:2])):
                strategy_num = 1
                trial = rand_to_best_2_bin(a, b, c, d, mut, min_b, max_b, popj, dimensions, best, cr)
            elif(rand_sp < sum(sp[:3])):
                strategy_num = 2
                trial = rand_2_bin(a, b, c, d, e, mut, min_b, max_b, popj, dimensions, cr)
            else:
                strategy_num = 3
                trial = current_to_rand_1_bin(a, b, c, popj, mut, min_b, max_b)
            f = fobj(trial)
            if f < fitness[j]:
                fitness[j] = f
                population[j] = trial
                cr_memory[strategy_num].append(cr)
                success_memory[i, strategy_num] += 1
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial
            else:
                failure_memory[i, strategy_num] += 1
                pass
            pass
        pass
    for i in range (its):
        success_sum = pd.DataFrame(success_memory).sum(axis = 0)
        failure_sum = pd.DataFrame(failure_memory).sum(axis = 0)
        skg = success_sum / failure_sum + 0.01
        sp = skg / sum(skg)
        success_memory[(i%lp)] = 0
        failure_memory[(i%lp)] = 0
        for j in range (popsize):
            popj = population[j]
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c, d, e = population[np.random.choice(idxs, 5, replace = False)]
            strategy_num = -1
            rand_sp = np.random.rand()
            mut = random.gauss(0.5, 0.3)
            if(rand_sp < sp[0]):
                strategy_num = 0
                cr_median_0 = statistics.median(cr_memory[0])
                cr_0 = random.gauss(cr_median_0, 0.1)
                if((cr < 0) or (cr > 1)):
                    cr_0 = random.gauss(cr_median_0, 0.1)
                trial = rand_1_bin(a, b, c, mut, min_b, max_b, popj, dimensions, cr_0)
            elif(rand_sp < sum(sp[:2])):
                strategy_num = 1
                cr_median_1 = statistics.median(cr_memory[1])
                cr_1 = random.gauss(cr_median_1, 0.1)
                if((cr < 0) or (cr > 1)):
                    cr_0 = random.gauss(cr_median_0, 0.1)
                trial = rand_to_best_2_bin(a, b, c, d, mut, min_b, max_b, popj, dimensions, best, cr_1)
            elif(rand_sp < sum(sp[:3])):
                strategy_num = 2
                cr_median_2 = statistics.median(cr_memory[2])
                cr_2 = random.gauss(cr_median_2, 0.1)
                if((cr < 0) or (cr > 1)):
                    cr_0 = random.gauss(cr_median_0, 0.1)
                trial = rand_2_bin(a, b, c, d, e, mut, min_b, max_b, popj, dimensions, cr_2)
            else:
                strategy_num = 3
                trial = current_to_rand_1_bin(a, b, c, popj, mut, min_b, max_b)
            f = fobj(trial)
            if f < fitness[j]:
                fitness[j] = f
                population[j] = trial
                cr_memory[strategy_num].append(cr)
                success_memory[i%lp, strategy_num] += 1
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial
            else:
                failure_memory[i%lp, strategy_num] += 1
        if(np.fabs(min(fitness) - goal) < 1e-6):
            print(i)
            break
        yield best, fitness[best_idx]
        pass
    pass


def rand_1_bin(a, b, c, mut, min_b, max_b, popj, dimensions, cr):
    mutant = np.clip(a + mut * (b - c), min_b, max_b)
    cross_points = np.random.rand(dimensions) < cr
    if not np.any(cross_points):
        cross_points[np.random.randint(0, dimensions)] = True
    trial = np.where(cross_points, mutant, popj)
    return trial


def rand_to_best_2_bin(a, b, c, d, mut, min_b, max_b, popj, dimensions, best, cr):
    mutant = np.clip(popj + mut * (best - popj) + mut * (a - b) + mut * (c - d), min_b, max_b)
    cross_points = np.random.rand(dimensions) < cr
    if not np.any(cross_points):
        cross_points[np.random.randint(0, dimensions)] = True
    trial = np.where(cross_points, mutant, popj)
    return trial


def rand_2_bin(a, b, c, d, e, mut, min_b, max_b, popj, dimensions, cr):
    mutant = np.clip(a + mut * (b - c) + mut * (d - e), min_b, max_b)
    cross_points = np.random.rand(dimensions) < cr
    if not np.any(cross_points):
        cross_points[np.random.randint(0, dimensions)] = True
    trial = np.where(cross_points, mutant, popj)
    return trial


def current_to_rand_1_bin(a, b, c, popj, mut, min_b, max_b):
    k = np.random.rand()
    trial = np.clip(popj + k * (a - popj) + mut * (b - c), min_b, max_b)
    return trial


def jade(fobj, bounds, popsize=20, its=1000, goal=0, c=0.5):
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    population = min_b + pop * diff
    population_new = np.random.rand(popsize, dimensions)
    for i in range(len(population_new)):
        population_new[i] = pop[i]
    mut = 0.5
    mean_cr = 0.5
    mean_mut = 0.5
    a = []
    for i in range(its):
        s_mut = []
        s_cr = []
        matrix_sort(population, fobj, pop)
        best = population[0]
        fitness_best = fobj(best)
        fitness = np.asarray([fobj(ind) for ind in population])
        for j in range(popsize):
            p = int(np.random.rand() * popsize)
            idx_x_best_p = random.randint(0, p)
            x_best_p = population[idx_x_best_p]
            idxs = [idx for idx in range(popsize) if idx != j]
            x_r1, x_r2 = population[np.random.choice(idxs, 2, replace=False)]
            idx_x_r2 = random.randint(0, len(population) + len(a) - 3)
            if idx_x_r2 >= (len(population) - 2):
                x_r2 = a[idx_x_r2 - len(population) + 2]
            mut = cauchy.rvs(loc=mean_mut, scale=0.1)
            while mut < 0 or mut > 1:
                mut = cauchy.rvs(loc=mean_mut, scale=0.1)
            mutant = np.clip(population[j] + mut * (x_best_p - population[j]) + mut * (x_r1 - x_r2), min_b, max_b)
            cr = random.gauss(mean_cr, 0.1)
            cross_points = np.random.rand(dimensions) < cr
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, population[j])
            f = fobj(trial)
            if f < fitness[j]:
                population_new[j] = trial
                a.append(population[j])
                s_cr.append(cr)
                s_mut.append(mut)
            else:
                population_new[j] = population[j]
        while len(a) > popsize:
            index = random.randint(0, len(a)-1)
            a.pop(index)
        for k in range(len(population_new)):
            population[k] = population_new[k]
        mean_cr = (1 - c) * mean_cr + c * np.mean(s_cr)
        mean_mut = (1 - c) * mean_mut + c * (sum(ff ** 2 for ff in s_mut) / sum(s_mut))
        if np.fabs(fitness_best - goal) < 1e-6:
            print(i)
            break
        yield best, fitness_best
        pass
    pass


def matrix_sort(matrix, fobj, pop):
    for i in range(len(matrix)):
        for j in range(i, len(matrix)):
            if fobj(matrix[i]) > fobj(matrix[j]):
                pop[i] = matrix[i]
                pop[j] = matrix[j]
                matrix[i] = pop[j]
                matrix[j] = pop[i]
    return matrix


def sacpmde(fobj, bounds, popsize=20, its=1000, goal = 0):
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    population = min_b + pop * diff
    fitness = np.asarray([fobj(ind) for ind in population])
    best_idx = np.argmin(fitness)
    best = population[best_idx]
    for i in range(its):
        for j in range(popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            select_idx = (np.random.choice(idxs, 3, replace = False)).tolist()
            fit_select = [fitness[i1] for i1 in select_idx]
            best1 = np.argmin(fit_select)
            others = [n for n in range(len(select_idx)) if n != best1]
            a_idx = select_idx[best1]
            b_idx = select_idx[others[0]]
            c_idx = select_idx[others[1]]
            mut = 0.1 + (0.9 - 0.1)*((fitness[b_idx] - fitness[a_idx]) / (fitness[c_idx] - fitness[a_idx]))
            mutant = np.clip(population[a_idx] + mut * (population[b_idx] - population[c_idx]), min_b, max_b)
            crossp = 0.1
            if(fitness[j] > np.mean(fitness)):
                crossp = 0.1 + (0.6 - 0.1)*(fitness[j] - min(fitness) / (max(fitness) - min(fitness)))
            cross_points = np.random.rand(dimensions) < crossp
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, population[j])
            f = fobj(trial)
            if f < fitness[j]:
                fitness[j] = f
                population[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial
        if(np.fabs(min(fitness) - goal) < 1e-6):
            print(i)
            break 
        yield best, fitness[best_idx]
        pass
    pass

In [6]:
def Rastrigin(x):
    return sum(x ** 2 - 10 * np.cos(2 * np.pi * x) + 10)
def Griewank(x):
    part = 1
    for i in range (len(x)):
        part *= np.cos(x[i]/(i+1) ** 0.5)
    return 1 - part + sum(x ** 2) / 4000

In [7]:
def rastrigin_de_test(mut, cr):
    result = []
    for num in range(20):
        it = list(de(Rastrigin, [(-5.12,5.12)] * 30,mut = mut, cr = cr, popsize = 100, its = 3000))
        result.append(it[-1][-1])
        pass
    mean_result = np.mean(result)
    std_result = np.std(result)
    success_num = 0
    for i in result:
        if np.fabs(i - 0) < 1e-5:
            success_num += 1
            pass
        pass
    return mean_result, std_result, success_num

In [9]:
mean_result, std_result, success_num = rastrigin_de_test(0.9, 0.1)
print(mean_result, std_result, success_num)

1623
1645
1638
1630
1657
1641
1631
1657
1628
1654
1633
1645
1632
1705
1645
1652
1649
1678
1653
1652
1.0859355453085584e-06 7.961438561228348e-08 20


In [11]:
mean_result, std_result, success_num = rastrigin_de_test(0.9, 0.9)
print(mean_result, std_result, success_num)

274.50890727464173 28.500257667802003 0


In [12]:
mean_result, std_result, success_num = rastrigin_de_test(0.5, 0.3)
print(mean_result, std_result, success_num)

41.14023062323036 2.994835704092035 0


In [17]:
def rastrigin_de_randtobest_1_test(mut, cr):
    result = []
    for num in range(20):
        it = list(de_randtobest_1(Rastrigin, [(-5.12,5.12)] * 30,mut = mut, cr = cr, popsize = 100, its = 3000))
        result.append(it[-1][-1])
        pass
    mean_result = np.mean(result)
    std_result = np.std(result)
    success_num = 0
    for i in result:
        if np.fabs(i - 0) < 1e-5:
            success_num += 1
            pass
        pass
    return mean_result, std_result, success_num
mean_result, std_result, success_num = rastrigin_de_randtobest_1_test(0.5, 0.3)
print(mean_result, std_result, success_num)

18.407912069096742 3.335152148206913 0


In [18]:
def rastrigin_de_randtobest_2_test(mut, cr):
    result = []
    for num in range(20):
        it = list(de_randtobest_2(Rastrigin, [(-5.12,5.12)] * 30,mut = mut, cr = cr, popsize = 100, its = 3000))
        result.append(it[-1][-1])
        pass
    mean_result = np.mean(result)
    std_result = np.std(result)
    success_num = 0
    for i in result:
        if np.fabs(i - 0) < 1e-5:
            success_num += 1
            pass
        pass
    return mean_result, std_result, success_num
mean_result, std_result, success_num = rastrigin_de_randtobest_2_test(0.5, 0.3)
print(mean_result, std_result, success_num)

50.2110295056436 3.965113014685203 0


In [13]:
def rastrigin_sade_test():
    result = []
    for num in range(20):
        it = list(sade(Rastrigin, [(-5.12,5.12)] * 30, popsize = 100, its = 3000))
        result.append(it[-1][-1])
        pass
    mean_result = np.mean(result)
    std_result = np.std(result)
    success_num = 0
    for i in result:
        if np.fabs(i - 0) < 1e-5:
            success_num += 1
            pass
        pass
    return mean_result, std_result, success_num
mean_result, std_result, success_num = rastrigin_sade_test()
print(mean_result, std_result, success_num)

2336
2244
2012
2559
1818
2313
2071
2476
2496
2171
2290
2271
1920
2065
2023
2066
0.3482366071441188 0.7881553840549694 16


In [14]:
def rastrigin_jde_test():
    result = []
    for num in range(20):
        it = list(jde(Rastrigin, [(-5.12,5.12)] * 30, popsize = 100, its = 3000))
        result.append(it[-1][-1])
        pass
    mean_result = np.mean(result)
    std_result = np.std(result)
    success_num = 0
    for i in result:
        if np.fabs(i - 0) < 1e-5:
            success_num += 1
            pass
        pass
    return mean_result, std_result, success_num
mean_result, std_result, success_num = rastrigin_jde_test()
print(mean_result, std_result, success_num)

11.461892001474709 4.217869585592278 0


In [15]:
def rastrigin_sacpmde_test():
    result = []
    for num in range(20):
        it = list(sacpmde(Rastrigin, [(-5.12,5.12)] * 30, popsize = 100, its = 3000))
        result.append(it[-1][-1])
        pass
    mean_result = np.mean(result)
    std_result = np.std(result)
    success_num = 0
    for i in result:
        if np.fabs(i - 0) < 1e-5:
            success_num += 1
            pass
        pass
    return mean_result, std_result, success_num
mean_result, std_result, success_num = rastrigin_sacpmde_test()
print(mean_result, std_result, success_num)

1621
1603
1662
1592
1577
1512
1611


c:\users\oliver\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:310: RuntimeWarning: divide by zero encountered in double_scalars


1602
1636
1610
1729
1603
1713
1693
1669
1715
1650
1689
1619
1699
1.0895887909256885e-06 8.094881144570976e-08 20
